# Exercise sheet 3

Please write the names of all group members below:

Names: Vali Florinel Craciun, Javokhir Isomurodov

Below you can find some imports that might be useful. The 'secrets' library includes a true random number generator (e.g. to generate a nonce). The 'ecdsa' library allows you to generate secret and public key pairs and to sign and verify methods. You can find the documentation here: https://pypi.org/project/ecdsa/

In [1]:
import json
import datetime
import hashlib
import secrets
import ecdsa
from ecdsa import VerifyingKey, SigningKey

Below you can find a short code demo on how to sign and verify messages with the ecdsa library. If you want to hash a dictionary that includes a secret/public key (or a signature), you need to convert them to a string first.

In [2]:
sk = SigningKey.generate(curve=ecdsa.SECP256k1)

#generate pk of sk
pk = sk.get_verifying_key()

#convert sk to bytes
pkpem = pk.to_pem()

#convert sk of type bytes to type string
pk_str=bytes.decode(pkpem)

#convert sk of type string back to type bytes
pk_bytes = bytes(pk_str, 'utf-8')

pk2 = VerifyingKey.from_pem(pk_bytes)

In [3]:
print('the statement sk == sk2 is', pk==pk2)

the statement sk == sk2 is True


In [4]:
#generate sk
sk = SigningKey.generate(curve=ecdsa.SECP256k1)

#generate pk of sk
pk = sk.get_verifying_key()

#convert sk to bytes
skpem = sk.to_pem()

#convert sk of type bytes to type string
sk_str=bytes.decode(skpem)

#Now the secret key is json serializable and can be stored in a dictionary
mydict = {
    'pk' : sk_str
}
bytes_dict = json.dumps(mydict, sort_keys=True).encode()

#convert sk of type string back to type bytes
sk_bytes = bytes(sk_str, 'utf-8')

#Now we have the original secret key again
sk2 = SigningKey.from_pem(sk_bytes)
print('the statement sk == sk2 is', sk==sk2)

# signature of bytes_dict, using the secret key
sig = sk2.sign(bytes_dict, hashfunc=hashlib.sha256)

#verify signature. We need the signature, the message (bytes_dict) and the public key 
pk.verify(sig, bytes_dict, hashfunc=hashlib.sha256)

the statement sk == sk2 is True


True

## Exercise 1 (20 points + 3 BONUS points)

Implement the class blockchain(). The class has several methods:

1. create_identity()            (1 point)
2. coin_creation_transaction()  (3 points)
3. create_transaction()         (3 points)
4. proof_of_work()              (3 points)
5. verify_signature()           (3 points)
6. prevent_double_spending()    (3 points)
7. create_block()               (4 points)
8. verify_block()               (3 BONUS points)

You are free to change the inputs/arguments of the methods, if it helps you to solve the exercise.

In [6]:
class blockchain():
    def __init__(self):
        self.prev_block_hash = 'Genesis block' 
        self.chain = []
        self.mempool = []
        
    def create_identity(self):
        '''
        create a new identity with a secret and public key pair (sk,pk). Return (sk,pk).
        Use the ecdsa library
        '''
        
        sk = SigningKey.generate(curve=ecdsa.SECP256k1)
        pk = sk.get_verifying_key()
        
        return sk, pk
 
    def coin_creation_transaction(self, pk, coinbase=str):
        '''
        Create a coinbase/coin creation transaction of type dictionary. A coinbase transaction does 
        not redeem a previous output, and it has a null hash pointer indicating this (a 256 bit string
        of '0's). It has a coinbase parameter which can contain arbitrary data.
        
        The value of the coinbase transaction is the block reward (we don't take transaction fees into
        account in this exercise), which is set to 25.
        The transaction also contains some meta data, such as the number of inputs and outputs and the
        hash of the transaction (hash the input and output of the transaction, not the metadata)
        
        'pk' = public key. Needed to redeem the coins of the coinbase transaction in another 
                           transaction 
        'coinbase' = This can be arbitrary data, such as a string
        '''
        
        pkpem = pk.to_pem()
        pk_str=bytes.decode(pkpem)
        
        transaction = {
            'num_inputs': 0,
            'num_outputs': 1,
            'hash': '',  # Place holder
            'inputs': [{
                'prev_out': {
                    'hash': '0'*64
                },
                'coinbase': coinbase
            }],
            'outputs': [{
                'value': '25',  # Block reward
                'pk': pk_str,
            }]
        }
        
        transaction['hash'] = self.calculate_transaction_hash(transaction)  # Calling the newly created calculate_transaction_hash method
        
        return transaction
    
    
    def create_transaction(self, transaction_hash, sk, outputs):
        '''
        create a transaction of type dictionary and sign it. The new transaction is stored in 
        self.mempool (the memory pool)
        
        inputs: (in theory, there can be multiple inputs to a transaction (multiple other transactions
                 that are redeemed in the new transaction), we are going to restrict ourselves to one 
                 input in this exercise)
            transaction_hash = The hash of the transaction that is redeemed in this transaction 
            sk = secret key. Required to sign the transaction
            
        outputs: All outputs are stored in a list to allow for multiple coin recipients. For each 
            recipient, you should pass a 'value' and 'pk'.
            value = output value of the transaction
            pk = public key. Needed to redeem the coins of the transaction in another transaction
            
        A transaction also contains meta data such as the number of inputs and outputs and the hash of 
        the transaction (hash of the inputs and outputs)
        '''  
        
        transaction = {
            'num_inputs': 1,
            'num_outputs': len(outputs),
            'hash': '',  # Placeholder, actual hash logic needed
            'inputs': [{
                'prev_out': {
                    'hash': transaction_hash
                },
                'signature': '' # Placeholder for signature
            }],
            'outputs': outputs
        }
        
        # Calculate the hash of the transaction
        transaction['hash'] = self.calculate_transaction_hash(transaction)
        
        # converting the types for json serializable
        for i, j in enumerate(transaction['outputs']):
            pk_bytes = bytes(j['pk'], 'utf-8')
            pk2 = VerifyingKey.from_pem(pk_bytes)
            transaction['outputs'][i]['pk'] = pk2

        # Sign the transaction
        signature = sk.sign(transaction['hash'].encode())
        transaction['inputs'][0]['signature'] = signature.hex()
        
        # Add the signed transaction to the mempool
        self.mempool.append(transaction)
    
    def calculate_transaction_hash(self, transaction):  # new method to calculate hash for each transaction
        '''
        Calculating the hash of a transaction based on its inputs and outputs.
        Implementing the actual logic.
        '''
        
        transaction_encoded = json.dumps(transaction, sort_keys=True).encode() 
        transaction_hash = hashlib.sha256(transaction_encoded).hexdigest()
        
        return transaction_hash
            
    def proof_of_work(self, block:dict):
        '''
        Generate a nonce with a true random number generator (you can use the secrets library) and 
        hash the block with the nonce. The block hash should have a specified amount of leading zero 
        bits (this depends on your computer, try out 3-5 leading zero bits). 
        
        block = A block (type dictionary), that is hashed with the nonce
        '''
        
        target_zeros = 3  # Placeholder for the required leading zeros
        nonce = secrets.randbits(32)  # Using secrets library for true random number generation
        block['nonce'] = nonce
        
        block_hash  = self.hash_meets_target(block, target_zeros)
        while block_hash == None:
            nonce = secrets.randbits(32)
            block['nonce'] = nonce
            block_hash  = self.hash_meets_target(block, target_zeros)
        
        block['hash'] = block_hash
    
    def hash_meets_target(self, block: dict, target_zeros: int):
        '''
        Check if the hash of the block meets the target number of leading zeros.
        '''
        block_encoded = json.dumps(block, sort_keys=True).encode()
        block_hash = hashlib.sha256(block_encoded).hexdigest()
        if block_hash.startswith('0' * target_zeros):
            return block_hash

    def verify_signature(self, transaction:dict):
        '''
        Verify, if the signature in the transaction is valid. The signature is valid, if it verifies 
        under the public key, that is specified in the output part of the transaction that is spend
        here.
        
        transaction = A transaction (as created in create_transaction()) of type dictionary
        '''
        
        sig = transaction['inputs'][0]['signature']
        pk = transaction['outputs'][0]['pk']
        
        transaction_encoded = json.dumps(transaction, sort_keys=True).encode()
        
        if pk.verify(sig, transaction_encoded, hashfunc=hashlib.sha256) == True:
            return 'valid'
        return 'invalid signature'
        
        
    def prevent_double_spending(self, transaction:dict):
        '''
        Verify, that the transaction is not a double spend. 
        Each input of a transaction specifies a previous transaction (it contains a hash of that 
        transaction). This hash should not have been redeemed in another transaction that is already 
        enclosed in a block.
        
        transaction = A transaction (as created in create_transaction()) of type dictionary
        '''
        
        current_hash = transaction['inputs'][0]['prev_out']['hash']
        for tx in self.mempool:
            for j in tx['inputs']['prev_out']['hash']:
                if current_hash == j['pk']:
                    return 'double spending'
                
        return 'valid'
            
    
    def create_block(self, pk, coinbase):
        '''
        Create a new block and append it to self.chain. A block is of type dictionary.
        The block encloses all transactions that are currently stored in self.mempool and then deletes
        them from self.mempool (a transaction should be removed from the memory pool, once it is 
        enclosed in a block). Store the transactions in a list (we don't store them in a merkle tree 
        to reduce the complexity of the programming assignment).
        
        Each block also includes the hash (no pointer!) of the previous block (stored in 
        self.prev_block_hash, the first block uses the 'Genesis block' hash already stored there), 
        a timestamp, a nonce, and the number of transactions.
        
        Before a transaction is enclosed in a block, the transaction should be verified (valid 
        signature, no double spending).
        
        The person/identity that creates the new block is allowed to create a coinbase transaction. 
        Use the public key of that person as output to the coin creation transaction (pk is the
        recipient of the coinbase transaction).
        
        Apply proof-of-work. The number of leading zeros of the block hash should be between 3-5 
        (this is up to you and should depend on how fast your computer is)
        
        pk = public key. Used for the coinbase transaction
        coinbase = This can be arbitrary data. Used for the coinbase transaction
        '''
        
        transaction = self.coin_creation_transaction(pk, coinbase)
        
        all_transactions = [transaction] + [tx for tx in self.mempool
                                            if self.verify_signature(tx) == self.prevent_double_spending(tx)]  # verifying the tx
        
        self.mempool.clear()
        
        block = {
            'transactions': all_transactions,
            'previous_hash': self.prev_block_hash,
            'timestamp': str(datetime.time()),
            'nonce': 0, # Placeholder
            'n_tx': len(all_transactions)
        }
        
        self.proof_of_work(block)  # Proof of work and hashing
        
        self.chain.append(block)
        
        return block
    
    def verify_block(self, block:dict):
        '''
        Verify, if a block is valid. A block is valid, if:
            there is only one coin creation transaction, 
            each transaction is valid (no double spending and correct signatures), 
            the block hash should have the predefined number of leading zero bits
        
        block: The block that should be verified
        '''
        msg = ''
        cnt = 0
        for tx in block:
            if tx['inputs'][0]['prev_out']['hash'] == '0' * 64:
                cnt += 1
                if cnt > 1:
                    msg =+'more than one coin creation transaction/ '
        
        verify = self.verify_signature(tx)
        double_spend = self.prevent_double_spending(tx)
        if not verify == double_spend:
            msg = msg + verify + '/ ' + double_spend
        
        if not block['hash'].startswith('0' * target_zeros):
            msg += 'no proof of work'
        
        return msg if msg else 'valid'

## Exercise 2 (5 BONUS points in total)

In this exercise, we are going to implement a little use case, using the newly implemented class blockchain(). (5 BONUS points)

1. Use the class blockchain() to create 5 identities (sk,pk):  Alice, Alice_2, Bob, Charlie  and Dave.
2. In total, we are going to create 3 blocks: Alice creates the first block (with a coinbase transaction) and is now the owner of 25 coins.
3. Alice wants to send 10 coins to Bob. Create a valid transaction that sends 10 coins to Bob and 15 coins back to her. The transaction should be stored in the memory pool.
4. Charlie creates the second block and encloses the valid transaction from Alice to Bob (plus a coinbase transaction) in the block. Charlie now owns 25 coins
5. Alice would like to double spend her coins. Create a transaction that sends all 25 coins (from the coinbase transaction that is stored in the first block) to Alice_2, which is also an identity of Alice. This transaction should be rejected and not be stored in the third block.
6. Dave would like to spend Charlies coins and send them to Bob. Create a transaction, in which Charlies coins are send to Bob. Sign the transaction with Dave's secret key. The transaction should be rejected, because the signature should not verify.
7. Charlie creates the third block. This block should only contain the coinbase transaction, because the other 2 transactions were rejected


In [7]:
bc = blockchain()

In [8]:
# 1

alice = bc.create_identity()
alice_2 = bc.create_identity()
bob = bc.create_identity()
charlie  = bc.create_identity()
dave = bc.create_identity()

In [9]:
# 2

bc.create_block(alice[1], 'first block')

{'transactions': [{'num_inputs': 0,
   'num_outputs': 1,
   'hash': 'afc379b115dbc30fa6c8936c124b0cb29fdc4d95f0bea78964d86d6f10e846b5',
   'inputs': [{'prev_out': {'hash': '0000000000000000000000000000000000000000000000000000000000000000'},
     'coinbase': 'first block'}],
   'outputs': [{'value': '25',
     'pk': '-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEJgiG8yugeHoAI6wM8l9aYy9AmeQqlTmS\nxwz0N1kQcaxi7I2zFeQSKUyFVH0CacYg5XOTOASz5r5ew1BF1JPKUw==\n-----END PUBLIC KEY-----\n'}]}],
 'previous_hash': 'Genesis block',
 'timestamp': '00:00:00',
 'nonce': 3747724784,
 'n_tx': 1,
 'hash': '000ea9ef0462f4e3d425faad7393acd0a47207114a978645eba1f104197d359e'}

In [10]:
# 3

bc.create_transaction('6a3d31acb1efd19839825ca6fdf4f9a963cc8b20b990d308e92d2721320f5d41', alice[0], outputs=[
    {
        'value': '10',
        'pk': bytes.decode(bob[1].to_pem())
    },
    {
        'value': '15',
        'pk': bytes.decode(alice[1].to_pem())
    }
])

In [ ]:
# 4

bc.create_block(charlie[1], 'second block')